In [57]:

## Get All interface function name
import asyncio
from dbus_next.aio import MessageBus
from dbus_next.constants import MessageType, BusType
async def main():
    bus = await MessageBus(bus_type=BusType.SYSTEM).connect()
    try:
        introspection = await bus.introspect("org.freedesktop.NetworkManager", "    ")
        obj = bus.get_proxy_object("org.freedesktop.NetworkManager", "/org/freedesktop/NetworkManager", introspection)
        interface = obj.get_interface("org.freedesktop.NetworkManager")
        intr_interface = next(i for i in obj.introspection.interfaces if i.name == "org.freedesktop.NetworkManager")
        print([interface._to_snake_case(i.name) for i in intr_interface.signals])
        print([interface._to_snake_case(i.name) for i in intr_interface.properties])
        print([interface._to_snake_case(i.name) for i in intr_interface.methods])
    except Exception as e:
        print(f"Error during introspection or signal handling: {e}")
        return

await main()

['check_permissions', 'state_changed', 'device_added', 'device_removed']
['devices', 'all_devices', 'checkpoints', 'networking_enabled', 'wireless_enabled', 'wireless_hardware_enabled', 'wwan_enabled', 'wwan_hardware_enabled', 'wimax_enabled', 'wimax_hardware_enabled', 'radio_flags', 'active_connections', 'primary_connection', 'primary_connection_type', 'metered', 'activating_connection', 'startup', 'version', 'version_info', 'capabilities', 'state', 'connectivity', 'connectivity_check_available', 'connectivity_check_enabled', 'connectivity_check_uri', 'global_dns_configuration']
['reload', 'get_devices', 'get_all_devices', 'get_device_by_ip_iface', 'activate_connection', 'add_and_activate_connection', 'add_and_activate_connection2', 'deactivate_connection', 'sleep', 'enable', 'get_permissions', 'set_logging', 'get_logging', 'check_connectivity', 'state', 'checkpoint_create', 'checkpoint_destroy', 'checkpoint_rollback', 'checkpoint_adjust_rollback_timeout']


In [17]:
import asyncio
from dbus_next.aio import MessageBus
from dbus_next.message_bus import Message
from dbus_next.constants import MessageType, BusType

def print_message(message: Message):
#    if message.message_type == MessageType.SIGNAL:
    print(vars(message))
    return False

async def main():
    bus = await MessageBus(bus_type=BusType.SYSTEM).connect()
#    introspection = await bus.introspect("org.freedesktop.NetworkManager", "/org/freedesktop/NetworkManager/Devices/1")
#    print(introspection.tostring())

    bus.add_message_handler(print_message)

    # Keep the script running
    try:
        while True:
            await asyncio.sleep(1)
    except (KeyboardInterrupt, asyncio.CancelledError):
        print("Script interrupted by user")
        bus.disconnect()

if __name__ == "__main__":
    await main()


<!DOCTYPE node PUBLIC "-//freedesktop//DTD D-BUS Object Introspection 1.0//EN"
"http://www.freedesktop.org/standards/dbus/1.0/introspect.dtd">
<node>
  <interface name="org.freedesktop.DBus.Properties">
      <method name="Get">
          <arg name="interface_name" direction="in" type="s" />
            <arg name="property_name" direction="in" type="s" />
            <arg name="value" direction="out" type="v" />
        </method>
        <method name="GetAll">
          <arg name="interface_name" direction="in" type="s" />
            <arg name="properties" direction="out" type="a{sv}" />
        </method>
        <method name="Set">
          <arg name="interface_name" direction="in" type="s" />
            <arg name="property_name" direction="in" type="s" />
            <arg name="value" direction="in" type="v" />
        </method>
        <signal name="PropertiesChanged">
          <arg name="interface_name" direction="out" type="s" />
            <arg name="changed_properties" dire

In [84]:
import asyncio
from dbus_next.aio import MessageBus
from dbus_next.message_bus import Message
from dbus_next.constants import MessageType, BusType

async def network_added(network_path):
    print(f"Network added: {network_path}")
    # Add your logic here

async def network_removed(network_path):
    print(f"Network removed: {network_path}")
    # Add your logic here

async def state_changed(state):
    print(f"State changed: {state}")
    # Add your logic here

def device_state_changed(message: Message):
#    if message.message_type == MessageType.SIGNAL:
    if message.member == 'StateChanged':
        print(vars(message))

async def main():
    bus = await MessageBus(bus_type=BusType.SYSTEM).connect()

    try:
        introspection = await bus.introspect("org.freedesktop.NetworkManager", "/org/freedesktop/NetworkManager")
        obj = bus.get_proxy_object("org.freedesktop.NetworkManager", "/org/freedesktop/NetworkManager", introspection)
        interface = obj.get_interface("org.freedesktop.NetworkManager")
        interface.on_device_added(network_added)
        interface.on_device_removed(network_removed)
        interface.on_state_changed(state_changed)
        print("Listening for NetworkManager signals...")

        bus.add_message_handler(device_state_changed)
    except Exception as e:
        print(f"Error during introspection or signal handling: {e}")
        return

    # Keep the script running
    try:
        while True:
            await asyncio.sleep(1)
    except (KeyboardInterrupt, asyncio.CancelledError):
        print("Script interrupted by user")
        bus.disconnect()

if __name__ == "__main__":
    await main()


Listening for NetworkManager signals...
Script interrupted by user


In [5]:
import asyncio
from systemd import journal

j = journal.Reader(flags=journal.SYSTEM_ONLY)
j.log_level(journal.LOG_INFO)  # Adjust log level if needed
j.add_match(SYSLOG_IDENTIFIER="kernel")
j.seek_tail()
j.get_previous()

#j.log_level(journal.LOG_INFO)

async def process(event):
    print(event)
    print(event["MESSAGE"])

def callback():
    print("got a callback, processing")
    j.process()
    for entry in j:
        asyncio.ensure_future(process(entry))

asyncio.get_running_loop().add_reader(j.fileno(), callback)
try:
    while True:
        await asyncio.sleep(1)
except (KeyboardInterrupt, asyncio.CancelledError):
    print("Script interrupted by user")
    asyncio.get_running_loop().remove_reader(j.fileno())
    j.close()


got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
got a callback, processing
Script interrupted by user


In [12]:
import asyncio
from systemd import journal

async def listen_to_journal():
    j = journal.Reader()
    j.log_level(journal.LOG_WARNING)  # Adjust log level if needed
    j.add_match(SYSLOG_IDENTIFIER="kernel") # Only match kernel log
    j.this_boot()  # Only show logs from this boot

    # Move to the end of the journal so we only get new messages
    j.seek_tail()
    j.get_previous()

    loop = asyncio.get_running_loop()

    try:
        while True:
            # Wait for new messages
            await loop.run_in_executor(None, j.wait, 1)

            # Process new messages
            for entry in j:
                if entry['MESSAGE'].startswith("CIFS: VFS:") and "has not responded" in entry['MESSAGE']:
                    print(f"{entry['__REALTIME_TIMESTAMP']}: {entry['MESSAGE']}")

            # Sleep for a bit to avoid busy waiting
            await asyncio.sleep(0.5)
    except (KeyboardInterrupt, asyncio.CancelledError):
        print("Script interrupted by user")
        j.close()


await listen_to_journal()


2024-07-13 17:28:50.734516+02:00: CIFS: VFS: \\fn-hmsrv-001-storage.gdkprod.com has not responded in 3 seconds. Reconnecting...
2024-07-13 17:29:17.947396+02:00: CIFS: VFS: \\fn-hmsrv-001-storage.gdkprod.com has not responded in 3 seconds. Reconnecting...
Script interrupted by user
